In [1]:
from pynq import Overlay
import numpy as np
import random
from pynq import allocate

In [2]:
overlay = Overlay('/home/xilinx/pynq/overlays/aes_axis/aes.bit')
overlay?

In [3]:
aes_ip = overlay.aes_0
aes_ip?

In [4]:
dma = overlay.axi_dma
dma_send = overlay.axi_dma.sendchannel
dma_recv = overlay.axi_dma.recvchannel

In [5]:
data_size = 16

key_and_plaintext_buffer = allocate(shape=(2*data_size,), dtype=np.uint32)
key_array = [ord(char) for char in "kkkkeeeeyyyy...."]
plaintext_array = [ord(char) for char in "abcdef1234567890"]
key_and_plaintext_buffer[:] = key_array + plaintext_array

ciphertext_and_decryptedtext_buffer = allocate(shape=(2*data_size,), dtype=np.uint32)

In [6]:
CONTROL_REGISTER = 0x0
#aes_ip.write(CONTROL_REGISTER, 0x01) # Set just AP_START to 1
aes_ip.write(CONTROL_REGISTER, 0x81) # Set AP_START and AUTO_RESTART to 1
aes_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  key_size = Register(key_size=write-only)
}

In [7]:
aes_ip.write(0x10, 16) # Write Key Size = 16
aes_ip.read(0x10)

16

In [8]:
def run_kernel():
    dma_send.transfer(key_and_plaintext_buffer)
    dma_recv.transfer(ciphertext_and_decryptedtext_buffer)
    dma_send.wait()
    dma_recv.wait()

In [9]:
run_kernel()

In [10]:
key_and_plaintext_buffer

PynqBuffer([107, 107, 107, 107, 101, 101, 101, 101, 121, 121, 121, 121,
             46,  46,  46,  46,  97,  98,  99, 100, 101, 102,  49,  50,
             51,  52,  53,  54,  55,  56,  57,  48], dtype=uint32)

In [11]:
ciphertext_and_decryptedtext_buffer

PynqBuffer([107, 107, 107, 107, 101, 101, 101, 101, 121, 121, 121, 121,
             46,  46,  46,  46,  97,  98,  99, 100, 101, 102,  49,  50,
             51,  52,  53,  54,  55,  56,  57,  48], dtype=uint32)

In [12]:
# Print out ciphertext_buffer
for i in range(16):
    print('0x' + format(ciphertext_and_decryptedtext_buffer[i], '02x'))
# Print out decryptedtext_buffer
for i in range(16):
    print('0x' + format(ciphertext_and_decryptedtext_buffer[i+16], '02x'))

0x6b
0x6b
0x6b
0x6b
0x65
0x65
0x65
0x65
0x79
0x79
0x79
0x79
0x2e
0x2e
0x2e
0x2e
0x61
0x62
0x63
0x64
0x65
0x66
0x31
0x32
0x33
0x34
0x35
0x36
0x37
0x38
0x39
0x30


In [13]:
del key_and_plaintext_buffer
del ciphertext_and_decryptedtext_buffer